In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [2]:
max_features = 1000
maxlen = 10

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

model = Sequential([
    Embedding(max_features, 32),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.2)

loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/5
157/157 [==============================] - 5s 14ms/step - loss: 0.6272 - accuracy: 0.6341 - val_loss: 0.5797 - val_accuracy: 0.6854
Epoch 2/5
157/157 [==============================] - 1s 7ms/step - loss: 0.5444 - accuracy: 0.7168 - val_loss: 0.5710 - val_accuracy: 0.6944
Epoch 3/5
157/157 [==============================] - 1s 7ms/step - loss: 0.5088 - accuracy: 0.7436 - val_loss: 0.5889 - val_accuracy: 0.6854
Epoch 4/5
157/157 [==============================] - 1s 6ms/step - loss: 0.4755 - accuracy: 0.7692 - val_loss: 0.6027 - val_accuracy: 0.6718
Epoch 5/5
782/782 [==============================] - 2s 2ms/step - loss: 0.6432 - accuracy: 0.6748
Test Accuracy: 0.6747599840164185


In [4]:
def predict_sentiment(text, model, maxlen=maxlen):
    tokens = imdb.get_word_index()
    tokens = {k: (v + 3) for k, v in tokens.items()}
    tokens["<PAD>"] = 0
    tokens["<START>"] = 1
    tokens["<UNK>"] = 2
    tokens["<UNUSED>"] = 3

    reverse_tokens = dict([(value, key) for (key, value) in tokens.items()])

    sequence = [tokens.get(word, tokens["<UNK>"]) for word in text.split()]
    sequence = pad_sequences([sequence], maxlen=maxlen)

    prediction = model.predict(sequence)[0, 0]
    sentiment = "positive" if prediction >= 0.5 else "negative"
    return sentiment, prediction

In [5]:
text1 = "This movie was great! I really enjoyed it."
text2 = "The acting was terrible and the plot was boring."
sentiment1, score1 = predict_sentiment(text1, model)
sentiment2, score2 = predict_sentiment(text2, model)

print(f"Text: '{text1}' | Sentiment: {sentiment1} | Score: {score1:.4f}")
print(f"Text: '{text2}' | Sentiment: {sentiment2} | Score: {score2:.4f}")

1/1 [==============================] - 0s 30ms/step
Text: 'This movie was great! I really enjoyed it.' | Sentiment: positive | Score: 0.5840
Text: 'The acting was terrible and the plot was boring.' | Sentiment: negative | Score: 0.0727
